<img style="float: right; height:90px" src="dove_air.png"> 

 # Dove Air Site Location Analysis

<div style="text-align: right">Author: Phil Li</div>
<div style="text-align: right"><b>Engineering and Data Analyst</b></div>

In [1]:
# Python Libraries

In [1]:
import numpy as np
import descartes
import pandas as pd
import matplotlib.pyplot as pp
#%matplotlib inline
%matplotlib widget
import contextily as ctx
from math import radians, cos,sin,asin,sqrt
# pd.set_option('display.max_columns',30)
import ipywidgets as widgets
from ipywidgets import GridspecLayout
import os,sys
import geopandas as gpd
import ipyvuetify as v
import psycopg2
import os,sys 
import urllib.parse as urlparse
import warnings
def ignore_warn(*args,**kwargs):
    pass
warnings.warn=ignore_warn

In [3]:
os.chmod("/opt/conda/lib/python3.7/site-packages/.wh.conda-4.8.2-py3.7.egg-info",0o755)
os.chmod("/opt/conda/lib/python3.7/site-packages/.wh.certifi-2020.6.20-py3.7.egg-info",0o755)
os.chmod("/opt/conda/lib/python3.7/site-packages/.wh.certifi-2020.6.20-py3.6.egg-info",0o755)
os.chmod("/opt/conda/lib/python3.7/site-packages/.wh.certifi-2019.11.28-py3.7.egg-info",0o755)

In [4]:
import geoplot as gpt
import mapclassify 

In [7]:
hospital_partial_df=pd.read_csv('Site_location_programing.csv')
hospital_gdf=gpd.GeoDataFrame(hospital_partial_df,geometry=gpd.points_from_xy(hospital_partial_df.longitude,hospital_partial_df.latitude))

In [8]:
## Functions Created
#Functions created to make the code more readable and compact

In [33]:
def main_plot(df_map,df_facility,ax):
    """
    plot the background map
    """
    scheme=mapclassify.Quantiles(df_facility.population, 7)
    gpt.choropleth(df_map,hue=df_map.population,cmap='PuBu',alpha=0.3,edgecolor='k',scheme=scheme,legend=True,legend_labels=['<319k', '319k-346k', '346k-444k', '444k-514k', '514k-531k','531k-610k','>610k'],legend_kwargs={'bbox_to_anchor': (0.87, 1.05),'fontsize':'small'},ax=ax)
    ctx.add_basemap(ax=ax,crs='EPSG:4326', source=ctx.providers.OpenStreetMap.Mapnik)
    df_facility[df_facility.facility_type=='HOSPITAL'].plot(ax=ax,markersize=90, color='r',marker='v',label='Hospital',legend=True)
    df_facility[df_facility.facility_type=='CLINIC'].plot(ax=ax,markersize=25,color='k',marker=',',label='Clinic',legend=True)
    df_facility[(df_facility.facility_type!='HOSPITAL') | (df_facility.facility_type!='CLINIC')].plot(ax=ax,markersize=10, color='k',marker='.',label='Others')

def change_coordinates(facility_name,x,y):
    """
    change facility coordinates in medical_df
    """
    medical_gdf.loc[medical_gdf.name==facility_name,'latitude']=x
    medical_gdf.loc[medical_gdf.name==facility_name,'longitude']=y
    facility=medical_gdf.loc[medical_gdf.name==facility_name]
    facility=gpd.GeoDataFrame(facility,geometry=gpd.points_from_xy(facility.longitude,facility.latitude))
    
def service_range(hosp):
    """
    return 150km range circle & Marker circle by giving a hospital/clinics name
    """
    hospital=medical_gdf[medical_gdf.name==hosp]
    circle_l=hospital.geometry.buffer(1.39)
    circle_s=hospital.geometry.buffer(0.07)
    return circle_l,circle_s

def station_location_plot(ax,*vars):
    """
    Plot a single range circle and a center name
    """
    # 'Palegreen','blueviolet','aqua'
    color=pd .DataFrame({0:'aqua',1:'Palegreen',2:'blueviolet',3:'tomato'},index=['a'])
    for i,hosp in enumerate(vars):
        circle_l,circle_s=service_range(hosp)
        circle_l.plot(ax=ax,alpha=0.5,edgecolor='k',linewidth=1,color=color[i].values[0])
        circle_s.plot(ax=ax,alpha=0.5,edgecolor='k',linewidth=1,color='r')
       
        #plot hospital & region names
        hospital=medical_gdf.loc[medical_gdf.name==hosp]
        pp.text(hospital.geometry.x, hospital.geometry.y, f'{hosp}, {hospital.district.values[0]}',fontsize=8,fontweight='bold')

def distance_cal(hospital_n,section_n):
    """"
    calculate the distance between two points on the earth
    """
    #conver decimal degrees to radians
    sec=population_cal_df[population_cal_df['section']==section_n]
    hosp=medical_gdf[medical_gdf.name==hospital_n]
    lon1,lat1,lon2,lat2=map(radians,[sec.longitude,
                            sec.latitude,
                            hosp.longitude,
                            hosp.latitude])
    #haversing formula
    dlon=lon2-lon1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c=2*asin(sqrt(a))
    r=6371
    return round(c*r,2)

site_population={}

def population_cal(hospital_n):
    """"
    calculate the population covered within the circle
    """
    pop={}
    for hosp in hospital_n:
        pop[hosp]=0
        for section in population_cal_df.section:
            dis=distance_cal(hosp,section)
            if dis>=145 and dis<153:
                pop[hosp]=pop[hosp]+0.5*population_cal_df[population_cal_df['section']==section].population.values[0]
            elif dis<145:
                pop[hosp]=pop[hosp]+population_cal_df[population_cal_df['section']==section].population.values[0]
            else:
                continue
        pop[hosp]=round(pop[hosp])
        
    site_population.clear()
    for loc,population in pop.items():
        site_population[loc]=population   
#     print(f'                            The population coverage at {loc}: {population}')


def plot_graph(ax,dist):
    """
    Plot the texts and markers on the medical facilities' coordinates
    """
    chiefdom=chiefdom_population_df[chiefdom_population_df['chiefdom']==dist]
    city=hospital_gdf[hospital_gdf['district']==chiefdom.district.values[0]]
    ax=ax
    # Create a boundary
    boundary=list(city.total_bounds)
    # Only include population points within the boundary
    blob=chiefdom_population_df[chiefdom_population_df['district']==chiefdom.district.values[0]].reset_index(drop=True)
    blob['within']=0
    for i in blob.index:
        if boundary[3]>blob.iloc[i].latitude>boundary[1] and boundary[2]>blob.iloc[i].longitude>boundary[0]:
            blob.iloc[i,-1]=1
        else:
            blob.iloc[i,-1]=0
    # Create a background 
    west,south,east,north=(city.total_bounds)
    ghent_img, ghent_ext = ctx.bounds2img(west,
                                     south,
                                     east,
                                     north,
                                     ll=True,
                                     source=ctx.providers.OpenStreetMap.Mapnik,
                                    )
    img,ext=ctx.warp_tiles(ghent_img,extent=ghent_ext)
    ax.imshow(img, extent=ext)
    # Create population blobs
    gpt.pointplot(blob[blob['within']==1],scale='total',limits=(1,20),color='cornflowerblue',alpha=0.5,ax=ax) 
    
    # Hospital locations
    city[city['Note']!='City center'].plot(ax=ax,marker='*',color='r')
    city[city['Note']=='City center'].plot(ax=ax,marker='.',color='mediumblue')
    # Hospital names
    for i in city.index:
        x=hospital_gdf.iloc[i].geometry.x
        y=hospital_gdf.iloc[i].geometry.y
        pp.text(x,y,hospital_gdf.iloc[i].Name, fontsize=7, fontweight='bold',color='k')

In [34]:
# Connect to PostgreSQL

In [41]:
# conn=psycopg2.connect(host='localhost',database='sierra_geo',user='postgres')

url=urlparse.urlparse(os.environ['DATABASE_URL'])
dbname=url.path[1:]
user=url.username
password=url.password
host=url.hostname
port=url.port

conn=psycopg2.connect(dbname=dbname,
                    user=user,
                    password=password,
                    host=host,
                    port=port)

In [42]:
population_cal_df=pd.read_sql("SELECT * from population_cal",conn)
district_info_df=gpd.read_postgis("SELECT district_name,region_name,population,polygon AS geometry FROM district",conn,geom_col='geometry',crs=4326)
medical_gdf=gpd.read_postgis("SELECT district,name,address,type AS facility_type, population,latitude,longitude,point AS geometry FROM medical",conn,geom_col='geometry',crs=4326)
chiefdom_population_df=gpd.read_postgis("SELECT district,chiefdom,section,total,latitude,longitude,point AS geometry FROM chiefdom_population",conn,geom_col='geometry',crs=4326)
eez_shape_df=gpd.read_postgis("SELECT line_name,line AS geometry FROM eez",conn,geom_col='geometry',crs=4326)

In [43]:
conn.close()

In [21]:
city_select=v.Select(
    prepend_icon='mdi-home-city',
    style_='width: 290px',
    v_model='e7',
    items=['Bo Town','Kenema City'],
    label='City Name',
    chips=True    
    )
hospital_select=v.Select(
    prepend_icon='mdi-hospital-box',
    style_='width: 290px',
#    v_model='e6',
    items=['Tonkolili','Bo','Port Loko','Karene'],
    label='District Name (1)',
    multiple=True,
    chips=True
    ) 
hospital_EEZ_select=v.Select(
    prepend_icon='mdi-hospital-box',
    style_='width: 290px',
#    v_model='e6',
    items=['Tonkolili','Bo','Port Loko','Karene'],
    label='District Name (2)',
    multiple=True,
    chips=True
    )

In [ ]:
progress_1=v.ProgressCircular(             
    size=20,
    width=2,
    color='success',
    indeterminate=True,
)
progress_2=v.ProgressCircular(             
    size=20,
    width=2,
    color='deep-purple accent-4',
    indeterminate=True,
)
progress_3=v.ProgressCircular(             
    size=20,
    width=2,
    color='indigo darken-3',
    indeterminate=True,
)

In [26]:
west,south,east,north=(-18.297687,4.683876,-9.915118,9.999282)
ghent_img, ghent_ext = ctx.bounds2img(west,
                                     south,
                                     east,
                                     north,
                                     ll=True,
                                     source=ctx.providers.OpenStreetMap.Mapnik,
                                    )
img,ext=ctx.warp_tiles(ghent_img,extent=ghent_ext)

## Analysis Examples
Some analysis results are illustrated below. This includes nationwide medical resourses distribution map (NMRS), citywide medical resources distribution map (CMRS), Economic Exclusive Zone (EEZ) map
* `Colors` in NMRS and the `sizes` of circles in CMRS represent the population in that region
* The population coverage result underneath the NMRS figure is not accurate at the moment as more data is needed
* The circles in NMRS and EEZ have a radius of 150km



In [44]:
output_2=widgets.Output()
output_1=widgets.Output()
output_progress_1=widgets.Output()
output_progress_2=widgets.Output()
output_progress_3=widgets.Output()
with output_1:
    fig,ax=pp.subplots(figsize=(8,8))

select_show_1=v.Html(tag='div', class_='d-flex flex-row', children=[
    v.Html(tag='div', class_='d-flex flex-column', children=[
        #v.Html(tag='h3', children=['Country Map']),
        v.Chip(class_='ma-2',
              color='success',
              outlined=True,
              children=['City Map']),
        city_select,
    ]),
    v.Html(tag='div', class_='d-flex flex-column', children=[
        v.Chip(class_='ma-2',
              color='deep-purple accent-4',
              outlined=True,
              children=['Country Map']),
        hospital_select
    ]),
    v.Html(tag='div', class_='d-flex flex-column', children=[
        v.Chip(class_='ma-2',
              color='indigo darken-3',
              outlined=True,
              children=['Maritime']),
        hospital_EEZ_select
    ]),
])

select_show_2=v.Html(tag='div', class_='d-flex flex-row', children=[
    output_progress_1,output_progress_2,output_progress_3])
    
def on_select(widget,event,data):
        pp.cla()
        var=data
        if widget.label=='City Name':
            with output_progress_1:
                display(progress_1)
            output_2.clear_output()
            plot_graph(ax,var)
            output_progress_1.clear_output()
        elif widget.label=='District Name (1)':
            with output_progress_2:
                display(progress_2)
            var=pd.Series(var).map({'Tonkolili':'Lion Heart Hospital','Bo':'Kindoya Hospital','Port Loko':'Port Loko Government Hospital','Karene':'Holy Spirit Catholic Hosp'}).tolist()
            population_cal(var)
            output_2.clear_output()
            with output_2: 
                #print('\n')
                for key,value in site_population.items():
                    print(f'                              {key} population coverage: {value}')
            main_plot(district_info_df,medical_gdf,ax)
            station_location_plot(ax,*var)
            output_progress_2.clear_output()
        else:
            with output_progress_3:
                display(progress_3)
            output_2.clear_output()
            var=pd.Series(var).map({'Tonkolili':'Lion Heart Hospital','Bo':'Kindoya Hospital','Port Loko':'Port Loko Government Hospital','Karene':'Holy Spirit Catholic Hosp'}).tolist()
            ax.imshow(img, extent=ext)
            eez_shape_df.plot(ax=ax,color='r')
            station_location_plot(ax,*var)
            output_progress_3.clear_output()

city_select.on_event('change', on_select)
hospital_select.on_event('change',on_select)
hospital_EEZ_select.on_event('change',on_select)

# fig,ax=pp.subplots(figsize=(8,8))
# v.Container(children=[select_show])
    
grid = GridspecLayout(50, 16, height='1130px')
grid[:5, 2:14] = select_show_1
grid[3, 1] = select_show_2
grid[6:40, 2:14] =output_1
grid[40:50, 2:14] = output_2
grid

GridspecLayout(children=(Html(children=[Html(children=[Chip(children=['City Map'], class_='ma-2', color='succe…

/anaconda3/envs/Sierra_leone/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/anaconda3/envs/Sierra_leone/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/anaconda3/envs/Sierra_leone/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/anaconda3/envs/Sierra_leone/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to r